<a href="https://colab.research.google.com/github/AliaksandrSiarohin/articulated-animation/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Demo for paper "Motion Representations for Articulated Animation"
**Clone repository. First install git lfs. Pulling all checkpoints may take a while**


In [14]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")


In [15]:
# source_image = imageio.imread('sup-mat/source.png')
# source_image = imageio.imread('../data/ood_image_sample.png')
source_image = imageio.imread('../data/421162d871_waxflower_front.jpg')
# driving_video = imageio.mimread('sup-mat/driving.mp4')
driving_video = imageio.mimread('data/fashion_vids/train/91-wIan2z6S.mp4', memtest=False)



# ckpt_path = 'checkpoints/ted384.pth'
# ckpt_path = 'checkpoints/taichi256.pth'
# config_path = 'config/ted384.yaml'
# config_path = 'config/taichi256.yaml'
# size = (384, 384)
ckpt_path = '/Users/john.lim/Downloads/33999.pth'
config_path = 'config/fashion.yaml'
size = (256, 256)

**Load source image and driving video.**

In [16]:
source_image = resize(source_image, size)[..., :3]
driving_video = [resize(frame, size )[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Load checkpoints.**

In [17]:
from demo import load_checkpoints

generator, region_predictor, avd_network = load_checkpoints(config_path=config_path,
                                                            checkpoint_path=ckpt_path, cpu=True)

**Perform animation.**

In [18]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, 
                             region_predictor, avd_network, animation_mode='avd', cpu=True)

#save resulting video
imageio.mimsave('generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 346/346 [07:17<00:00,  1.27s/it]
